In [1]:
import os
import json
import codecs
import numpy as np
import tensorflow as tf
import yaml
import DM_process_v1 as DM_process

os.environ["CUDA_VISIBLE_DEVICES"] = '1'
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
tf.__version__

'2.3.0'

In [2]:
params = {
    'batch_size':16,
    'lr' : 0.001,
    'epochs': 160,
    'drops' : [0.1]
         }

In [3]:
with open('./DM_char.json', mode='r', encoding='utf-8') as f:
    dicts = json.load(f)

In [4]:
path = '../stories/all_stories.yml'
with open(path, 'r', encoding='utf-8') as f:
    dataset = yaml.load(f.read(),Loader=yaml.Loader)

In [5]:
action2id = dicts['action2id']
intent2id = dicts['intent2id']
slots2id = dicts['entities2id']
id2action = dicts['id2action']
id2intent = dicts['id2intent']
id2slots = dicts['id2entities']

In [6]:
previous_action_len = len(action2id)
# print(previous_action_len)
slots_len = len(slots2id)
# print(slots_len)
user_intent_len = len(intent2id)
# print(user_intent_len)

In [7]:
data_set = DM_process.split_data(dataset)

In [8]:
previous_action, slots, user_intent, action = DM_process.extract_conv_data(data_set,action2id,slots2id,intent2id)

In [9]:
def Dataset(previous_action_inputs, slots_inputs,user_intent_inputs,pre_action):
    dataset = tf.data.Dataset.from_tensor_slices(({
    "previous_action_inputs" : previous_action_inputs,
    "slots_inputs" : slots_inputs,
    "user_intent_inputs" : user_intent_inputs
    },
    {
        "pre_action" : pre_action
    }))
    data_count = len(slots_inputs)
    dataset = dataset.shuffle(buffer_size = data_count)
    dataset = dataset.batch(params['batch_size'])
    return dataset

In [10]:
train_dataset =  Dataset(previous_action, slots, user_intent, action)

In [11]:
tf.keras.backend.clear_session()
previous_action_inputs = tf.keras.layers.Input(shape=(previous_action_len,), name = 'previous_action_inputs')
slots_inputs = tf.keras.layers.Input(shape = (slots_len,), name = 'slots_inputs')
user_intent_inputs = tf.keras.layers.Input(shape = (user_intent_len,), name = 'user_intent_inputs')

previous_action_embed = tf.keras.layers.Embedding(128,64)(previous_action_inputs)
slots_embed = tf.keras.layers.Embedding(128,64)(slots_inputs)
user_intent_embed = tf.keras.layers.Embedding(128,64)(user_intent_inputs)

utter_inputs = tf.keras.layers.concatenate([previous_action_embed,slots_embed,user_intent_embed],axis=1)
bilstm = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128,return_sequences=True))(utter_inputs)
x_in = tf.keras.layers.LayerNormalization()(bilstm)
x_conv = tf.keras.layers.GlobalAveragePooling1D()(x_in)
pre_action = tf.keras.layers.Dense(previous_action_len, activation='softmax',name = 'pre_action')(x_conv)
model = tf.keras.Model([previous_action_inputs,slots_inputs,user_intent_inputs],pre_action)
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
previous_action_inputs (InputLa [(None, 35)]         0                                            
__________________________________________________________________________________________________
slots_inputs (InputLayer)       [(None, 13)]         0                                            
__________________________________________________________________________________________________
user_intent_inputs (InputLayer) [(None, 42)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 35, 64)       8192        previous_action_inputs[0][0]     
_______________________________________________________________________________________

In [12]:
losses = {'pre_action': 'categorical_crossentropy'}
metrics = {'pre_action': ['accuracy']}
optimizer = tf.keras.optimizers.Adam(params['lr'])
model.compile(optimizer, loss=losses, metrics=metrics)

In [13]:
file_path = '../DM_model_weight/DM_weight.h5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(file_path,
                                                        save_weights_only=False, save_best_only=True)
learning_rate_reduction = tf.keras.callbacks.ReduceLROnPlateau(patience=50, factor=0.5)
callbacks_list = [checkpoint,learning_rate_reduction]

In [14]:
# model.fit(x=[previous_action, slots, user_intent],y=action,batch_size=8,epochs=params['epochs'],callbacks=callbacks_list)

In [15]:
model.fit(train_dataset,epochs=params['epochs'])

Epoch 1/160
14/14 [==============================] - 0s 12ms/step - loss: 3.2718 - accuracy: 0.1577
Epoch 2/160
14/14 [==============================] - 0s 11ms/step - loss: 3.1055 - accuracy: 0.1622
Epoch 3/160
14/14 [==============================] - 0s 10ms/step - loss: 3.1257 - accuracy: 0.1577
Epoch 4/160
14/14 [==============================] - 0s 10ms/step - loss: 3.0822 - accuracy: 0.1847
Epoch 5/160
14/14 [==============================] - 0s 11ms/step - loss: 3.0617 - accuracy: 0.1847
Epoch 6/160
14/14 [==============================] - 0s 15ms/step - loss: 3.0667 - accuracy: 0.1847
Epoch 7/160
14/14 [==============================] - 0s 18ms/step - loss: 3.0440 - accuracy: 0.1757
Epoch 8/160
14/14 [==============================] - 0s 14ms/step - loss: 3.0486 - accuracy: 0.1577
Epoch 9/160
14/14 [==============================] - 0s 13ms/step - loss: 3.0415 - accuracy: 0.1712
Epoch 10/160
14/14 [==============================] - 0s 9ms/step - loss: 3.0182 - accuracy: 0.1712


In [15]:
# model.save_weights('../DM_model_weight/DM_weight_629.h5')

In [30]:
model.save(file_path)